## Uavs Collision

Collision of 2 uavs


In [1]:
import numpy as np

# Custom modules
from helpers import global2local,plot_3d_interactive, kill_processes
from simulators import Simulator,Gazebo,QGC

from plan import Plan,State
from gcs import GCS



kill_processes()

## Create Plans

In [2]:
offsets = [(-20, 0, 0, 0),(20, 0, 0, 0)]#
n_vehicles=len(offsets)


homes=np.array([offset[:3] for offset in offsets])
global_paths=[np.array([[-20, 0, 5],[20, 0, 5]]),
              np.array([[20, 0, 5],[-20, 0, 5]])]
local_paths=[global2local(path, home,pairwise=True) for path,home in zip(global_paths,homes)]
plans=[Plan.basic(wps=path,alt=5,wp_margin=0.5,navegation_speed=10) for path in local_paths]

## Visualization Parameters

In [3]:
waypoints = {f'waypoint_{i+1}': {'pos':global_paths[0],'color':'blue'} for i in range(n_vehicles)}
plot_3d_interactive(waypoints,title='Simulation Markers',expand=[0.2,0.2,0.6],ground=-0.05)

## Choose Simulator

In [4]:
simulator = Simulator(name="NONE",offsets=offsets,plans=plans)

# simulator=QGC(offsets=offsets,
#               plans=plans,
#               origin=(-35.3633245,149.1652241,0,0))

# simulator=Gazebo(offsets=offsets,
#                  plans=plans,
#                  world_path='simulators/gazebo/worlds/runway.world',
#                  vehicle_models=n_vehicles*['drone'],
#                  markers=waypoints)

Launch Simulator

In [5]:
uavs = simulator.launch()

ℹ️  Running without a simulator.
Vehicle 1 launched 🚀
Vehicle 2 launched 🚀


## Execute Plan

In [6]:
gcs=GCS(positions=homes.copy())
while gcs.incomplete_missions:
    to_remove = set()
    for id in gcs.incomplete_missions:
        uav = uavs[id-1]
        if uav.plan.state == State.DONE:
            to_remove.add(id)
        else:
            uav.plan.act()
    gcs.update_missions(to_remove)
kill_processes()

Vehicle 1: ▶️ Starting Step: Check disarmed
Vehicle 2: ▶️ Starting Step: Check disarmed
Vehicle 1: ✅ Step: Check disarmed is done
Vehicle 2: ✅ Step: Check disarmed is done
Vehicle 1: ▶️ Starting Step: Check EKF
Vehicle 2: ▶️ Starting Step: Check EKF
Vehicle 2: ✅ Step: Check EKF is done
Vehicle 2: ▶️ Starting Step: Check GPS
Vehicle 1: ✅ Step: Check EKF is done
Vehicle 1: ▶️ Starting Step: Check GPS
Vehicle 2: ✅ Step: Check GPS is done
Vehicle 2: ▶️ Starting Step: Check system
Vehicle 1: ✅ Step: Check GPS is done
Vehicle 1: ▶️ Starting Step: Check system
Vehicle 2: ✅ Step: Check system is done
Vehicle 2: ▶️ Starting Step: Switch to GUIDED
Vehicle 2: ✅ Step: Switch to GUIDED is done
Vehicle 2: ▶️ Starting Step: Set speed to 10.00 m/s
Vehicle 2: ✅ Step: Set speed to 10.00 m/s is done
Vehicle 2: ▶️ Starting Step: arm
Vehicle 1: ✅ Step: Check system is done
Vehicle 1: ▶️ Starting Step: Switch to GUIDED
Vehicle 1: ✅ Step: Switch to GUIDED is done
Vehicle 1: ▶️ Starting Step: Set speed to 10.

In [7]:
plans[0]

✅ <Plan 'basic'>
  ✅ <Action 'Pre-Arm Check'>
    ✅ <Step 'Check disarmed'>
    ✅ <Step 'Check EKF'>
    ✅ <Step 'Check GPS'>
    ✅ <Step 'Check system'>
  ✅ <Action 'Set Mode: GUIDED'>
    ✅ <Step 'Switch to GUIDED'>
  ✅ <Action 'Set Navigation Speed'>
    ✅ <Step 'Set speed to 10.00 m/s'>
  ✅ <Action 'Arm'>
    ✅ <Step 'arm'>
  ✅ <Action 'takeoff'>
    ✅ <Step 'takeoff'>
  ✅ <Action 'fly'>
    ✅ <Step 'go to  -> (0, 0, 5)'>
    ✅ <Step 'go to  -> (40, 0, 5)'>
  ✅ <Action 'Land'>
    ✅ <Step 'land'>

In [8]:
plans[1]

✅ <Plan 'basic'>
  ✅ <Action 'Pre-Arm Check'>
    ✅ <Step 'Check disarmed'>
    ✅ <Step 'Check EKF'>
    ✅ <Step 'Check GPS'>
    ✅ <Step 'Check system'>
  ✅ <Action 'Set Mode: GUIDED'>
    ✅ <Step 'Switch to GUIDED'>
  ✅ <Action 'Set Navigation Speed'>
    ✅ <Step 'Set speed to 10.00 m/s'>
  ✅ <Action 'Arm'>
    ✅ <Step 'arm'>
  ✅ <Action 'takeoff'>
    ✅ <Step 'takeoff'>
  ✅ <Action 'fly'>
    ✅ <Step 'go to  -> (0, 0, 5)'>
    ✅ <Step 'go to  -> (-40, 0, 5)'>
  ✅ <Action 'Land'>
    ✅ <Step 'land'>